<a href="https://colab.research.google.com/github/Maxindrull2/UFV_Visualicaci-n-de-datos/blob/main/ejercicios%20de%20clase/clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 1: Ventas en Europa**

Se descargan las librerias que se van a requerir

In [ ]:
!pip install plotly_express

Se importan las librerias necesarias

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import linregress
import plotly.graph_objects as go
import re

Se carga la base de datos

In [ ]:
df = pd.read_csv('datos_ejercicio_ventas.csv')

df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


Los significados de las columnas de la base de datos son los siguientes:

* COUNTRY: Pais

* SUBBRAND: Marca

* YEAR: Año de la venta, u objetivo de la predicción

* MONTH: Mes de la venta, u objetivo de la predicción

* SCENARIO:
  * actual --> Venta real
  * AI_forecast --> Predicción hecha por inteligencia artificial

* FORECAST: Mes en que se realizo la predicción. El mes es indicado por el número (AI_PF = Enero = 1)

* FORECAST_YEAR: Año en que se hizo la predicción

* AMOUNT: Cantidad de producto vendido

*Nota: Pese a que en clase se informó de que el formato del FORECAST era el mes+1. Ese formato da multiples errores en los datos, debido a que numerosos registros hacen predicciones hacia el pasado, lo cual carece de sentido. Por ello se ha optado realizarlo de esta manera, pues así se conserva la adecuación lógica necesaria.*

# **Limpieza general**

En este apartado se realizará una limpieza general de la base de datos, preparando sus características al formato adecuado necesario para todo el conjunto de ejercicios.

Se analiza la estructura del amount

In [ ]:
print(df['AMOUNT'].describe())

count    1.866600e+04
mean     9.721822e+05
std      1.915283e+06
min     -2.171201e+05
25%      8.754541e+04
50%      3.081759e+05
75%      1.078576e+06
max      1.481563e+07
Name: AMOUNT, dtype: float64


Los valores negativos no tienen sentido puesto que trata de la cantidad de ventas, por ello se procede a eliminarlos

In [ ]:
print("Hay", len(df[df['AMOUNT'] < 0]), "elmentos con valor negativo")
df = df[df['AMOUNT'] >= 0]

Hay 10 elmentos con valor negativo


Unicamente se eliminan 10 elementos de la base de datos, por lo que no se pierde excesiva información.

Se analizan los posibles valores de las otras variables

In [ ]:
for col in df.columns:
  if col != 'AMOUNT':
    print(f"\n{col}: {df[col].unique()}")
    print(f"Número de valores únicos: {len(df[col].unique())}")


COUNTRY: ['Portugal' 'Great Britain' 'Spain' 'Hungary' 'Norway' 'Denmark'
 'Netherlands' 'Italy' 'Czech']
Número de valores únicos: 9

SUBBRAND: ['Lipton (L3)' 'Pepsi Max (L3)' '7up (L3)' 'Pepsi Regular (L3)'
 'Mountain Dew (L3)' '7up Free (L3)']
Número de valores únicos: 6

YEAR: [2023 2024 2025]
Número de valores únicos: 3

MONTH: [12  9  2  4  7 11  1  6 10  3  5  8]
Número de valores únicos: 12

SCENARIO: ['AI_forecast' 'actual']
Número de valores únicos: 2

FORECAST: ['AI_P02F' 'AI_P10F' 'AI_P09F' 'AI_P03F' 'AI_PF' 'AI_P11F' 'AI_P06F'
 'AI_P05F' 'AI_P07F' 'AI_P12F' 'AI_P08F' 'AI_P04F' nan]
Número de valores únicos: 13

FORECAST_YEAR: [2023.   nan]
Número de valores únicos: 2


Todas las clases dadas van acorde a la estructura necesaria

Confirmamos que no existan fechas incorrectas en los actuals (que aun no hayan sucedido)

In [ ]:
print(sorted(df[df['SCENARIO'] == 'actual'][df[df['SCENARIO'] == 'actual']['YEAR'] == 2024]['MONTH'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8]


Agosto del 2024 es la ultima fecha registrada como veridica. Por lo que la información en este ámbito es correcta.

# **Ejercicio 1 - Distribución de ventas**

## **Limpieza específica**

En este apartado se realizaran las transformaciones sobre los datos que unicamente son necesarias o facilitan la realización de este ejercicio.

Se observan el número de predicciones y el número de ventas reales

In [ ]:
print(df['SCENARIO'].value_counts())

SCENARIO
AI_forecast    17766
actual           890
Name: count, dtype: int64


Puesto que queremos analizar el comportamiento de las ventas, se cogen unicamente los actuals y no las predicciones (forecast).

In [ ]:
df_ventas = df[df['SCENARIO'] == 'actual']
df_ventas.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04


Los datos de forecast y forecast year deberian de ser NaN

In [ ]:
print(df_ventas['SCENARIO'].unique())
print(df_ventas['FORECAST'].unique())
print(df_ventas['FORECAST_YEAR'].unique())

['actual']
[nan]
[nan]


Las columnas scenario, forecast y forecast year ya no son necesarias debido a que todos los datos son iguales y hacen referencia a las predicciones, que ahora mismo no se estan tomando en cuenta. Por ello se eliminan de la base de datos

In [ ]:
df_ventas = df_ventas.drop(['SCENARIO' ,'FORECAST', 'FORECAST_YEAR'], axis=1)
df_ventas.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,1.885949e+05
278,Portugal,7up (L3),2023,3,2.934971e+05
279,Portugal,7up (L3),2023,10,3.484466e+05
280,Great Britain,7up Free (L3),2023,10,1.172553e+06
281,Norway,Pepsi Regular (L3),2023,10,3.784859e+04


Se transforman los meses y años a formato fecha

In [ ]:
df_ventas['DATE'] = pd.to_datetime(df_ventas['YEAR'].astype(str) + '-' + df_ventas['MONTH'].astype(str), format='%Y-%m', errors='coerce')

df_ventas = df_ventas.drop(['YEAR', 'MONTH'], axis=1)

df_ventas.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
277,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
278,Portugal,7up (L3),2.934971e+05,2023-03-01
279,Portugal,7up (L3),3.484466e+05,2023-10-01
280,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
281,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


## **Ventas por pais**

En este apartado se analizara como se comporta el número de ventas en función de cada uno de los paises.

In [ ]:
country_amount = df_ventas.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
country_amount['LOGAMOUNT'] = np.log(country_amount['AMOUNT'])

fig = px.treemap(country_amount,
                 path=['COUNTRY'],
                 values='AMOUNT',
                 title='Cantidad de ventas total y porcentual por cada país',
                 labels={'AMOUNT': 'Cantidad total', 'COUNTRY': 'País'},
                 color='LOGAMOUNT',
                 color_continuous_scale='Viridis')

fig.update_layout(coloraxis_colorbar=dict(title='Cantidad total (escala logaritmica)'))

fig.update_traces(textinfo="label+percent parent", textposition="middle center")
fig.update_traces(customdata=country_amount['AMOUNT'] / 1000000)
fig.for_each_trace(lambda t: t.update(texttemplate="%{label}<br>%{customdata:.0f}M<br>%{percentParent:.1%}"))

fig.show()

En este gráfico treemap se puede apreciar como claramente las ventas de L3 son muy superiores y sonsacan sus beneficios principalmente de Gran Bretaña, puesto que el 50% de sus ventas proceden de este sector.

España, en cambio, es en el que con diferencia menos ventas hay. Representando unicamente un 1.2% de las ventas totales y siendo por tanto poco diferencial sobre el total.

Entre los otros 7 paises su fuerza esta más equilibrada, rondando entre el 9.5% y el 5.2% de las ventas. Entre sus dos extremos hay una diferencia del doble, pero se pueden considerar del mismo grupo a nivel comercial debido a la gran diferencia con respecto al máximo y mínimo de ventas.

## **Ventas por mes y año**

En este apartado se analizara como se comporta el número de ventas en función del tiempo, es decir, de las fechas dadas.

In [ ]:
date_amount = df_ventas.groupby('DATE')['AMOUNT'].sum().reset_index()

fig = px.line(date_amount,
              x='DATE',
              y='AMOUNT',
              title='Cantidad de ventas en función de la fecha',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Cantidad total'},
              markers=True)
fig.update_traces(marker=dict(size=10)) # size corresponds to radius of the marker
fig.show()

Los datos recopilados son pocos como para encontrar patrones que se repitan de manera continua. Pero en base al tiempo dado se puede sacar el como los primeros meses del año, sobre todo enero y febrero, suelen tener menos ventas. Y que los meses posteriores asciende considerablemente el total.

## **Ventas por marca**

En este apartado se analizara como se comporta el número de ventas en función de cada una de las marcas internas de L3.

In [ ]:
subbrand_amount = df_ventas.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

subbrand_amount = subbrand_amount.sort_values('AMOUNT', ascending=True)

fig = px.bar(subbrand_amount,
             x='AMOUNT',
             y='SUBBRAND',
             orientation='h',
             title='Suma total de ventas por cada marca interna',
             labels={'AMOUNT': 'Cantidad total (escala logarítmica)', 'SUBBRAND': 'Marca'})

fig.update_xaxes(type="log")

fig.show()

En esta gráfica con escala logaritmica se puede apreciar contundentemente las principales diferencias entre las ventas en cada marca. Pepsi Max tiene un claro liderazgo con cientos de millones de ventas, mientras que Mountain Dew se queda muy por detras del resto sin apenas llegar a 10M.

Las otras 4 marcas, aunque no tan extremas como las anteriores dos, tambien estan bien diferenciadas. Mostrando como Pepsi regular es la segunda con mayor número de ventas, y tras de ella Lipton. Entre cada una de estas marcas hay una diferencia de varios cientos de miles de ventas, las cuales son cifras lo suficientemente altas como para tratar de manera diferencial su importancia.

# **Ejercicio 2 - Tendencia y estacionalidad**

En este ejercicio se estudiara detenidamente la tendencia y estacionalidad de dos casos particulares dados.

* **Tendencia**:
  
  La tendencia se refiere a la dirección general y sostenida de una serie temporal a lo largo del tiempo. Indica si los valores tienden a aumentar, disminuir o permanecer constantes en el largo plazo. La tendencia puede ser:

  * Positiva: Cuando los valores tienden a crecer con el tiempo.
  * Negativa: Cuando los valores tienden a disminuir con el tiempo.
  * Constante: Cuando no se observa un cambio general en la serie, manteniéndose en el mismo nivel.

  La tendencia puede ser lineal o no lineal y suele capturar cambios estructurales a largo plazo en los datos, como el crecimiento poblacional, la inflación, o la mejora de tecnologías.

* **Estacionalidad**:
  La estacionalidad es un patrón recurrente en la serie temporal que se repite en intervalos regulares, generalmente relacionados con estaciones, meses o días específicos. Los patrones estacionales suelen ser causados por factores externos que influyen de manera periódica en los datos, como el clima, festividades, o ciclos económicos.

  * La estacionalidad implica que los datos de ciertos períodos (por ejemplo, cada diciembre, cada lunes, etc.) tienen valores similares, mostrando variaciones predecibles en función del tiempo.

## **Pais con menos ventas (Spain)**

Se estudia la tendencia y estacionalidad del pais con menos ventas, que como observamos en el ejercicio anterior era España (Spain)

In [ ]:
spain_sales = df_ventas[df_ventas['COUNTRY'] == 'Spain'].groupby('DATE')['AMOUNT'].sum().reset_index()

# Cálculo de la tendencia
x = np.arange(len(spain_sales))
y = spain_sales['AMOUNT'].values
pendiente, intercepto, _, _, _ = linregress(x, y)
tendencia = pendiente * x + intercepto

# Creación del gráfico original
fig = px.line(spain_sales, x='DATE', y='AMOUNT', title='Tendencia y estacionalidad para España',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Cantidad de ventas'})
fig.update_traces(marker=dict(size=10))  # Ajuste del tamaño de los marcadores

# Añadimos la línea de tendencia como línea discontinua
fig.add_trace(go.Scatter(
    x=spain_sales['DATE'],
    y=tendencia,
    mode='lines',
    name='Línea de Tendencia',
    line=dict(color='red', dash='dash')  # Línea roja discontinua
))

# Mostrar el gráfico
fig.show()


A priori la recta de regresión que representa la tendencia de los datos nos indica que el número de ventas va en aumento. Sin embargo, el valor de su pendiente no es muy elevado, por lo que podria ser constante debido a errores de aproximación al tener pocas fechas registradas.

Con la estacionalidad pasa algo similar. No se tienen los suficientes datos como para contrastar las conclusiones. Más que indicar que sobre los datos dados, el número de ventas en invierno es bastante inferior al de verano.

## **Marca con más ventas**

Se estudia la tendencia y estacionalidad de la marca con más ventas, que como observamos en el ejercicio anterior era Pepsi Max (L3)

In [ ]:
Pepsi_Max_sales = df_ventas[df_ventas['SUBBRAND'] == 'Pepsi Max (L3)'].groupby('DATE')['AMOUNT'].sum().reset_index()

# Cálculo de la tendencia
x = np.arange(len(Pepsi_Max_sales))
y = Pepsi_Max_sales['AMOUNT'].values
pendiente, intercepto, _, _, _ = linregress(x, y)
tendencia = pendiente * x + intercepto

# Creación del gráfico original
fig = px.line(Pepsi_Max_sales, x='DATE', y='AMOUNT', title='Tendencia y estacionalidad para Pepsi Max',
              labels={'DATE': 'Fecha', 'AMOUNT': 'Cantidad de ventas'})
fig.update_traces(marker=dict(size=10))  # Ajuste del tamaño de los marcadores

# Añadimos la línea de tendencia como línea discontinua
fig.add_trace(go.Scatter(
    x=Pepsi_Max_sales['DATE'],
    y=tendencia,
    mode='lines',
    name='Línea de Tendencia',
    line=dict(color='red', dash='dash')  # Línea roja discontinua
))

# Mostrar el gráfico
fig.show()

A diferencia del caso anterior, aqui si se puede ver una clara tendendia ascendente en el número de ventas. Ademas, tambien se puede observar una estacionalidad a pesar de los pocos datos. Esta estacionalidad es trimestral, puesto que cada tres meses asciende, para luego descender de manera abrupta y llegar a un mínimo. Coincidiendo en los meses de Enero, Abril, Julio y Octubre.

# **Ejercicio 3 - Predicciones España**

## **Limpieza específica**

En este apartado se realizaran las transformaciones sobre los datos que unicamente son necesarias o facilitan la realización de este ejercicio.

Se convierte el valor de forecast a su correspondiente numérico en meses

In [ ]:
def extract_and_process_forecast(df):
    """
    Extracts numbers from the 'FORECAST' column, converts them to integers,
    and subtracts 1. Handles potential errors gracefully.
    """
    def process_forecast_value(val):
        if pd.isna(val):
            return val
        try:
            numbers = re.findall(r'\d+', val)
            if numbers:
                return int(numbers[0])
            else:
                return 1 # Or handle cases with no numbers as needed
        except (ValueError, IndexError):
            return None # or handle the error differently

    df['FORECAST'] = df['FORECAST'].apply(process_forecast_value)
    return df


df = extract_and_process_forecast(df)
df

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,2.0,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,10.0,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,9.0,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,10.0,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,3.0,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,10.0,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,7.0,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,5.0,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,2.0,2023.0,5.330634e+05


Se convierte al formato int

In [ ]:
df['FORECAST'] = pd.to_numeric(df['FORECAST'], errors='coerce').astype('Int64')
df['FORECAST_YEAR'] = pd.to_numeric(df['FORECAST_YEAR'], errors='coerce').astype('Int64')

Se cogen unicamente los valores de España

In [ ]:
spain_df = df[df['COUNTRY'] == 'Spain']
spain_df

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,9,2023,88501.980847
20,Spain,Pepsi Regular (L3),2023,12,AI_forecast,5,2023,134268.151080
25,Spain,Lipton (L3),2025,3,AI_forecast,11,2023,9702.217953
62,Spain,7up Free (L3),2024,1,AI_forecast,4,2023,70144.329753
68,Spain,7up (L3),2024,4,AI_forecast,2,2023,38882.921227
...,...,...,...,...,...,...,...,...
18599,Spain,7up Free (L3),2023,9,actual,<NA>,<NA>,75888.808279
18611,Spain,Pepsi Regular (L3),2024,1,AI_forecast,7,2023,107080.159342
18617,Spain,Pepsi Max (L3),2024,10,AI_forecast,7,2023,96728.475541
18653,Spain,Pepsi Regular (L3),2024,3,AI_forecast,9,2023,120652.827718


Se convierten los meses y años del forecast a formato fecha

In [ ]:
spain_df['DATE'] = pd.to_datetime(spain_df['YEAR'].astype(str) + '-' + spain_df['MONTH'].astype(str), format='%Y-%m', errors='coerce')
spain_df['DATE_FORECAST'] = pd.to_datetime(spain_df['FORECAST_YEAR'].astype(str) + '-' + spain_df['FORECAST'].astype(str), format='%Y-%m', errors='coerce')
spain_df = spain_df.drop(['YEAR', 'MONTH', 'FORECAST_YEAR', 'FORECAST'], axis=1)
spain_df

<ipython-input-116-4f558677d06f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-116-4f558677d06f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,COUNTRY,SUBBRAND,SCENARIO,AMOUNT,DATE,DATE_FORECAST
2,Spain,Pepsi Max (L3),AI_forecast,88501.980847,2023-12-01,2023-09-01
20,Spain,Pepsi Regular (L3),AI_forecast,134268.151080,2023-12-01,2023-05-01
25,Spain,Lipton (L3),AI_forecast,9702.217953,2025-03-01,2023-11-01
62,Spain,7up Free (L3),AI_forecast,70144.329753,2024-01-01,2023-04-01
68,Spain,7up (L3),AI_forecast,38882.921227,2024-04-01,2023-02-01
...,...,...,...,...,...,...
18599,Spain,7up Free (L3),actual,75888.808279,2023-09-01,NaT
18611,Spain,Pepsi Regular (L3),AI_forecast,107080.159342,2024-01-01,2023-07-01
18617,Spain,Pepsi Max (L3),AI_forecast,96728.475541,2024-10-01,2023-07-01
18653,Spain,Pepsi Regular (L3),AI_forecast,120652.827718,2024-03-01,2023-09-01


## **Efectividad respecto a la fecha exacta**

El primero de los análisis a realizar acerca de la efectividad de las predicciones sera con respecto a la fecha exacta. Esto quiere decir que se tomaran todas las predicciones hechas para un día, y se comparará su valor con el real obtenido. Independientemente de cuando fueron realizadas dichas predicciones. Como información adicional se realizara el análisis por cada producto

La siguiente función calcula la diferencia entre el valor predicho y su valor real

In [ ]:
new_df = pd.DataFrame(columns=['DATE', 'SUBBRAND', 'AMOUNT'])

for date in spain_df['DATE'].unique():
    for subbrand in spain_df['SUBBRAND'].unique():
        filtered_df = spain_df[(spain_df['DATE'] == date) & (spain_df['SUBBRAND'] == subbrand)]

        if 'actual' in filtered_df['SCENARIO'].values and 'AI_forecast' in filtered_df['SCENARIO'].values:
            actual_amount = filtered_df[filtered_df['SCENARIO'] == 'actual']['AMOUNT'].sum()

            # Subtract the actual amount from each AI_forecast row individually
            filtered_df.loc[filtered_df['SCENARIO'] == 'AI_forecast', 'AMOUNT'] -= actual_amount

            # Now aggregate and store, for each 'AI_forecast' entry, the difference
            for index, row in filtered_df[filtered_df['SCENARIO'] == 'AI_forecast'].iterrows():
                new_df = pd.concat([new_df, pd.DataFrame({'DATE': [date], 'SUBBRAND': [subbrand], 'AMOUNT': [row['AMOUNT']], 'DATE_FORECAST': [row['DATE_FORECAST']]})], ignore_index=True)

<ipython-input-117-e9f1cde81973>:15: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Teniendo en cuenta que si las predicciones fuesen correctas, su valor seria 0. Tomando 0 como la media de los valores, se calcula la desviación tipica del error para cada producto

In [ ]:
std_dev_by_subbrand = new_df.groupby('SUBBRAND')['AMOUNT'].std(ddof=0).reset_index()
std_dev_by_subbrand.rename(columns={'AMOUNT': 'Standard Deviation'}, inplace=True)
std_dev_by_subbrand = std_dev_by_subbrand.sort_values(by='Standard Deviation', ascending=False)

fig = px.bar(std_dev_by_subbrand,
             x='SUBBRAND',
             y='Standard Deviation',
             title='Desviación estandar del error de predicción por marca',
             labels={'SUBBRAND': 'MArca', 'Standard Deviation': 'Desviación estandar del número de ventas'})

fig.show()

En esta gráfica se observa como Pepsi Regular es a priori cuyas aproximaciones son menos certeras, con una desviación tipica de 35k de ventas mensuales. Sin embargo, esta teoria no seria del todo cierta, puesto que el número de ventas total de cada uno de los productos es muy diferente. Y a nivel unitario (o porcentual) su importancia cambia.

Por ello, se calcula la media de ventas mensuales por cada producto, para poder realizar una comparación porcentual de la desviación.

In [ ]:
actual_sales_by_subbrand = spain_df[spain_df['SCENARIO'] == 'actual'].groupby('SUBBRAND')['AMOUNT'].mean()
actual_sales_by_subbrand

,AMOUNT
SUBBRAND,
7up (L3),85885.574654
7up Free (L3),72414.413164
Lipton (L3),10615.081199
Pepsi Max (L3),109452.540159
Pepsi Regular (L3),128195.689701


In [ ]:
# Merge the two dataframes
merged_df = pd.merge(std_dev_by_subbrand, actual_sales_by_subbrand.reset_index(), on='SUBBRAND', how='left')
merged_df.rename(columns={'AMOUNT': 'AMOUNT'}, inplace=True)

# Calculate the percentage
merged_df['Percentage'] = (merged_df['Standard Deviation'] / merged_df['AMOUNT']) * 100

# Sort the values
merged_df = merged_df.sort_values('Percentage', ascending=False)

# Calculate the mean percentage
mean_percentage = merged_df['Percentage'].mean()

# Create the bar chart using plotly
fig = px.bar(merged_df,
             x='SUBBRAND',
             y='Percentage',
             title='Porcentaje de la desviación estandar del error de predicción por marca',
             labels={'SUBBRAND': 'Marca', 'Percentage': 'Porcentaje de la desviación estandar del número de ventas'},
            )

# Add a horizontal line at the mean percentage
fig.add_shape(
    type="line",
    line=dict(dash="dash"),
    x0=4.5,
    y0=mean_percentage,
    x1=-0.5,
    y1=mean_percentage,
    line_color="red"
)

fig.add_annotation(
    x=len(merged_df) / 2,  # Adjust x position for better placement
    y=mean_percentage,
    text="Media",
    showarrow=False,
    yshift=10  # Shift the text slightly above the line
)

fig.show()

A partir de este gráfico se puede observar como la desviación tipica de las predicciones (error) suele ser de un 20% sobre el valor real. Además, se tiene que las marcas Pepsi Regular y Lipton suelen ser más dificiles de predecir, fallando un 10% más que el resto.

## **Efectividad respecto a la distancia temporal**

Como segundo análisis se busca observar si afecta o no el tiempo en que fue realizada la predicción. Tomando como hipotesis inicial que cuanto más cercana a la fecha real fuese realizada, más cerca estara de acertar el valor correcto.

A continuación se calculan las diferencias temporales para obtener el tiempo de predicción (TIME_DIFF)

In [ ]:
filtered_df = new_df
filtered_df['TIME_DIFF'] = filtered_df['DATE'] - filtered_df['DATE_FORECAST']
filtered_df['TIME_DIFF'] = (filtered_df['TIME_DIFF'].dt.days / 30.44).round()  # Aproximación
filtered_df

,DATE,SUBBRAND,AMOUNT,DATE_FORECAST,TIME_DIFF
0,2023-12-01,Pepsi Max (L3),-16940.131387,2023-09-01,3.0
1,2023-12-01,Pepsi Max (L3),8759.424011,2023-07-01,5.0
2,2023-12-01,Pepsi Max (L3),13267.232120,2023-11-01,1.0
3,2023-12-01,Pepsi Max (L3),11914.196921,2023-07-01,5.0
4,2023-12-01,Pepsi Max (L3),12654.959810,2023-12-01,0.0
...,...,...,...,...,...
1552,2023-01-01,Pepsi Max (L3),10031.356460,2023-01-01,0.0
1553,2023-01-01,Pepsi Regular (L3),9126.132791,2023-01-01,0.0
1554,2023-01-01,Lipton (L3),-1241.334093,2023-01-01,0.0
1555,2023-01-01,7up Free (L3),6418.799868,2023-01-01,0.0


Se agrupan los elementos en función del tiempo de predicción para asi calcular posteriormente la desviación típica para cada plazo.

In [ ]:
std_dev_by_time_diff = filtered_df.groupby('TIME_DIFF')['AMOUNT'].std(ddof=0).reset_index()
std_dev_by_time_diff.rename(columns={'AMOUNT': 'Standard Deviation'}, inplace=True)
std_dev_by_time_diff = std_dev_by_time_diff.sort_values(by='Standard Deviation', ascending=False)

time_diff_counts = filtered_df['TIME_DIFF'].value_counts().reset_index()
time_diff_counts.columns = ['TIME_DIFF', 'Count']

std_dev_by_time_diff = pd.merge(std_dev_by_time_diff, time_diff_counts, on='TIME_DIFF', how='left')

std_dev_by_time_diff = std_dev_by_time_diff.sort_values(by='TIME_DIFF')

Se muestran los resultados

In [ ]:
fig = px.scatter(std_dev_by_time_diff,
                 x='TIME_DIFF',
                 y='Standard Deviation',
                 size='Count',
                 title='Desviación estandar de la predicción en función de la diferencia temporal',
                 labels={'TIME_DIFF': 'Diferencia temporal (months)', 'Standard Deviation': 'Desviación estandar del numero de ventas', 'Count': 'Nº de registros'},
                 size_max=20) # Increased size_max for larger bubbles
fig.show()

En esta gráfica en la que el tamaño muestra el número de registros se puede sacar la conclusión de que a mayor tiempo de predicción, mayor sera el error cometido por esta. Por lo que la hipotesis es verdadera.

Tambien se puede ver que la hipotesis no funciona verazmente con los tiempos más largos, esto se debe a que el número de registros no es lo suficientemente representativo (<30), lo que da a errores de aproximación grandes

También se puede observar que el horizonte de previsión empleado por la empresa L3 es de 18 etapas